In [5]:
using Pkg; Pkg.activate("/home/roscar/work/cfgrib-project/cfgrib.jl")

Activating environment at `~/work/cfgrib-project/cfgrib.jl/Project.toml`


In [6]:
using cfgrib

In [7]:
path = "/home/roscar/work/cfgrib-project/cfgrib.jl/tests/sample-data/era5-levels-members.grib"

"/home/roscar/work/cfgrib-project/cfgrib.jl/tests/sample-data/era5-levels-members.grib"

In [8]:
using DataStructures

In [25]:
using GRIB

## Types

In [9]:
struct Variable
    dimensions::Tuple{Vararg{String}}
    data::Array
    attributes::Dict{String, String}
end

In [11]:
Variable(("1", ), [1,2], Dict("A"=>"b")) == Variable(("1", ), [1,2], Dict("A"=>"b"))

false

## build_dataset_components

In [12]:
a = "original GRIB coordinate for key:" *
     "arstarst"
println(a)

original GRIB coordinate for key:arstarst


In [125]:
#      Args
index = cfgrib.FileIndex(path, cfgrib.ALL_KEYS)
read_keys = []
kwargs = Dict()
#      Default
errors = "warn"
encode_cf = ("parameter", "time", "geography", "vertical")
squeeze = true
log = missing
LOG=log
# read_keys
time_dims = ("time", "step");

In [14]:
typeof(index)

cfgrib.FileIndex

In [15]:
collect(values(index.header_values))

59-element Array{Array,1}:
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 [0]                                             
 [120]                                           
 [61]                                            
 ⋮                                               
 Missing[missing]                                
 SubString{String}["z", "t"]                     
 [0]                                             
 SubString{String}["instant"]                    
 [1]                                             
 [0]                   

In [16]:
fieldnames(cfgrib.FileIndex)

(:allowed_protocol_version, :grib_path, :index_path, :index_keys, :offsets, :header_values, :filter_by_keys)

In [17]:
dimensions = OrderedDict()
variables = OrderedDict()
filter_by_keys = isdefined(index, :filter_by_keys) ? index.filter_by_keys : Dict();

In [18]:
#  THIS SHOULD BE A FOR LOOP
param_id = index["paramId"][1]

129

In [19]:
var_index = cfgrib.filter(index, paramId=param_id);

## build_variable_components

https://github.com/ecmwf/cfgrib/blob/2f50a1ec0331be5ed7476aa4a0bf66fb2e388652/cfgrib/dataset.py#L433

### enforce_unique_attributes

In [20]:
function enforce_unique_attributes(index::cfgrib.FileIndex, attribute_keys::Array, filter_by_keys::Dict)
    attributes = OrderedDict()
    for key in attribute_keys
        values = index[key]

        if length(values) > 1
            throw("Attributes are not unique for $key: $(values)")
        end

        value = values[1]

        if !ismissing(value)
            attributes["GRIB_" * key] = value
        end
    end

    return attributes
end


enforce_unique_attributes (generic function with 1 method)

### encode_cf_first

In [21]:
function encode_cf_first(data_var_attrs::OrderedDict, encode_cf::Tuple{Vararg{String}}=("parameter", "time"), time_dims::Tuple{Vararg{String}}=("time", "step"))
    coords_map = cfgrib.ENSEMBLE_KEYS
    param_id = get(data_var_attrs, "GRIB_paramId", missing)
    data_var_attrs["long_name"] = "original GRIB paramId: $(param_id)"
    data_var_attrs["units"] = "1"

    if "parameter" in encode_cf
        if haskey(data_var_attrs, "GRIB_paramId")
            data_var_attrs["standard_name"] = data_var_attrs["GRIB_cfName"]
        end

        if haskey(data_var_attrs, "GRIB_name")
            data_var_attrs["long_name"] = data_var_attrs["GRIB_name"]
        end

        if haskey(data_var_attrs, "GRIB_units")
            data_var_attrs["units"] = data_var_attrs["GRIB_units"]
        end
    end
    
    if "time" in encode_cf
        if issubset(time_dims, cfgrib.ALL_REF_TIME_KEYS)
            append!(coords_map, time_dims)
        else
            throw("time_dims $(time_dims) is not a subset of $(cfgrib.ALL_REF_TIME_KEYS)")
        end
    else
        append!(coords_map, cfgrib.DATA_TIME_KEYS)
    end
    
    append!(coords_map, cfgrib.VERTICAL_KEYS)
    append!(coords_map, cfgrib.SPECTRA_KEYS)
    
    return coords_map
end

encode_cf_first (generic function with 3 methods)

In [71]:
function first(index::cfgrib.FileIndex)
    GribFile(index.grib_path) do f
        first_offset = index.offsets[1][2][1]
        seek(f, first_offset)
        return Message(f)
    end    
end

first (generic function with 1 method)

In [126]:
function build_geography_coordinates(
        index, encode_cf, errors, log=LOG
    )
    
    first_message = first(var_index)
    geo_coord_vars = OrderedDict()
    grid_type = cfgrib.getone(index, "gridType")
    
    if "geography" in encode_cf && grid_type in GRID_TYPES_DIMENSION_COORDS
        geo_dims = ("latitude", "longitude")
        geo_shape = (getone(index, "Ny"), getone(index, "Nx"))
        latitudes = first_message["distinctLatitudes"]
        geo_coord_vars["latitude"] = Variable(
            ("latitude",), latitudes, cfgrib.COORD_ATTRS["latitude"]
        )
        
        if latitudes[1] > latitudes[end]
            geo_coord_vars["latitude"].attributes["stored_direction"] = "decreasing"
        end

        geo_coord_vars["longitude"] = Variable(
            ("longitude",),
            first_message["distinctLongitudes"],
            cfgrib.COORD_ATTRS["longitude"],
        )
    elseif "geography" in encode_cf && grid_type in GRID_TYPES_2D_NON_DIMENSION_COORDS
        throw("unimplemented")
    else
        throw("unimplemented")
    end
    
    return geo_dims, geo_shape, geo_coord_vars
end

build_geography_coordinates (generic function with 2 methods)

In [127]:
build_geography_coordinates(var_index, encode_cf, errors, log)

(("latitude", "longitude"), (61, 120), OrderedDict{Any,Any}("latitude" => Variable(("latitude",), [90.0, 87.0, 84.0, 81.0, 78.0, 75.0, 72.0, 69.0, 66.0, 63.0  …  -63.0, -66.0, -69.0, -72.0, -75.0, -78.0, -81.0, -84.0, -87.0, -90.0], Dict("units" => "degrees_north","stored_direction" => "decreasing","long_name" => "latitude","standard_name" => "latitude")),"longitude" => Variable(("longitude",), [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0  …  330.0, 333.0, 336.0, 339.0, 342.0, 345.0, 348.0, 351.0, 354.0, 357.0], Dict("units" => "degrees_east","long_name" => "longitude","standard_name" => "longitude"))))

In [247]:
function build_variable_components(index, encode_cf=(), filter_by_keys=Dict(), log=LOG, errors="warn", squeeze=true, read_keys=[], time_dims=("time", "step"))
    data_var_attrs_keys = cfgrib.DATA_ATTRIBUTES_KEYS
    data_var_attrs_keys = [data_var_attrs_keys; get(cfgrib.GRID_TYPE_MAP, index["gridType"][1], [])]
    data_var_attrs_keys = [data_var_attrs_keys; read_keys]

    data_var_attrs = enforce_unique_attributes(var_index, data_var_attrs_keys, filter_by_keys)

    coords_map = encode_cf_first(data_var_attrs, encode_cf, time_dims)

    coord_name_key_map = Dict()
    coord_vars = OrderedDict()

    for coord_key in coords_map
        values = index[coord_key]
        if length(values) == 1 && ismissing(values[1])
            #  TODO: Add logging
            #  @warn "Missing from GRIB Stream $(coord_key)"
            continue
        end
        
        coord_name = coord_key

        if ("vertical" in encode_cf && coord_key == "level" && haskey(data_var_attrs, "GRIB_typeOfLevel"))
            coord_name = data_var_attrs["GRIB_typeOfLevel"]
            coord_name_key_map[coord_name] = coord_key
        end
        
        attributes = Dict(
            "long_name" => "original GRIB coordinate for key: $(coord_key)($(coord_name))",
            "units"     => "1",
        )
        
        merge!(attributes, copy(get(cfgrib.COORD_ATTRS, coord_name, Dict())))
        
        data = sort(values, rev=get(attributes, "stored_direction", "none") == "decreasing")
        dimensions = (coord_name, )
        
        if squeeze && length(values) == 1
            data = data[1]
            typeof(data) == Array ? nothing : data = [data]
            dimensions = ()
        end
        
        coord_vars[coord_name] = Variable(dimensions, data, attributes)
    end
    
    header_dimensions = Tuple(d for (d, c) in pairs(coord_vars) if !squeeze || length(c.data) > 1)
    header_shape = Tuple(size(coord_vars[d].data) for d in header_dimensions)
    
    geo_dims, geo_shape, geo_coord_vars = build_geography_coordinates(index, encode_cf, errors)
    
    dimensions = (header_dimensions..., geo_dims)
    shape = (header_shape..., geo_shape)
    
    merge!(coord_vars, geo_coord_vars)
    
    offsets = OrderedDict{NTuple{length(header_dimensions), Int64}, Int}()
    for (header_values, offset) in var_index.offsets
        header_indexes = Array{Int}(undef, length(header_dimensions))
        for (i, dim) in enumerate(header_dimensions)
            coord_name = get(coord_name_key_map, dim, dim)
            coord_idx = findfirst(var_index.index_keys .== coord_name)
            header_value = header_values[coord_idx]
            header_indexes[i] = findfirst(coord_vars[dim].data .== header_value)
        end
        
        offsets[Tuple(header_indexes)] = offset
    end
    
    return offsets
    
    return coord_name_key_map
end

build_variable_components (generic function with 8 methods)

In [248]:
_d = build_variable_components(var_index, encode_cf, filter_by_keys, log, errors, squeeze, read_keys, time_dims)

OrderedDict{Tuple{Int64,Int64,Int64},Int64} with 80 entries:
  (1, 1, 2)  => 0
  (2, 1, 2)  => 14760
  (3, 1, 2)  => 29520
  (4, 1, 2)  => 44280
  (5, 1, 2)  => 59040
  (6, 1, 2)  => 73800
  (7, 1, 2)  => 88560
  (8, 1, 2)  => 103320
  (9, 1, 2)  => 118080
  (10, 1, 2) => 132840
  (1, 1, 1)  => 295200
  (2, 1, 1)  => 309960
  (3, 1, 1)  => 324720
  (4, 1, 1)  => 339480
  (5, 1, 1)  => 354240
  (6, 1, 1)  => 369000
  (7, 1, 1)  => 383760
  (8, 1, 1)  => 398520
  (9, 1, 1)  => 413280
  (10, 1, 1) => 428040
  (1, 2, 2)  => 590400
  (2, 2, 2)  => 605160
  (3, 2, 2)  => 619920
  (4, 2, 2)  => 634680
  (5, 2, 2)  => 649440
  ⋮          => ⋮